In [ ]:
__author__ = 'andy'

import sys
sys.path.append('/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages')

import pdb
import numpy as np
import yahoo_finance as y
from pandas import *
import matplotlib.pyplot as plot
from multiprocessing import Pool
import itertools as i
import nitime.analysis.granger as gc
from nitime.viz import drawmatrix_channels
import statsmodels.tsa.stattools as stats
import statsmodels.api as sm
from scipy.fftpack import fft
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts

class MultiagentSystem(object):

    def __init__(self, start, end):
        self.startDate = start
        self.endDate = end

        #init stock sectors and other variables
        self.numAgents = 7
        self.numHighAgents = 2
        self.numLags = 5
        self.sectorsTxt = ['it','energy','financials','industrials','healthcare','consumerdisc','consumerstaples']
        self.it = ['CTSH','ADP','PYPL','YHOO','AVGO','EBAY','BRCM','GLW','MU','VMW']
        self.energy = ['PSX','APC','HAL','WMB','VLO','MPC','BHI','SE','DVN','APA']
        self.financials = ['CB','MMC','BBT','CME','STT','EQR','AFL','ALL','DFS','BEN']
        self.industrials = ['PCP','EMR','AAL','CSX','DE','ITW','ETN','NSC','WM','CMI']
        self.healthcare = ['SYK','BDX','HUM','VRTX','CAH','HCA','ILMN','BAX','MYL','BXLT']
        self.consumerdisc = ['CMCSK','TSLA','CBS','CCL','LVS','VIAB','FOX','TRI','DISH','VIA']
        self.consumerstaples = ['MDLZ','COST','CL','KHC','KMB','RAI','KR','GIS','ADM','EL']
        self.sectors = [self.it,self.energy,self.financials,self.industrials,self.healthcare,self.consumerdisc,self.consumerstaples]#,self.ipos]

        self.agentGetPricesRef=agentGetPricesRef

    def runAgents(self):
        agentPool = Pool(self.numAgents)

        sectors = self.sectors
        sectorsTxt = self.sectorsTxt
        startDate = self.startDate
        endDate = self.endDate
        numLags = self.numLags

        sectorPrices = agentPool.map(self.agentGetPricesRef,i.izip(sectors,sectorsTxt, i.repeat(self.startDate),i.repeat(self.endDate)))
        #sectorPrices2 = self.getSectorPrices(sectors,sectorsTxt,startDate,endDate)

        #print "sectorPrices: ", sectorPrices
        #print "sectorPrices2: ", sectorPrices2

        #if sectorPrices == sectorPrices2: print "the same"
        #else: print "not the same"

        fulldata=[]
        highcorrs,highpairs,highidxs,fulldata = self.findHighCorrs(sectors,sectorPrices,numLags,fulldata)

        print "highcorrs: ",highcorrs
        print "highpairs: ",highpairs
        print "highidxs: ",highidxs

        #self.getGrangers(sectors,sectorPrices,highcorrs,highpairs)

        #print highcorrs
        #print highpairs

        #printing all the highest correlations
        self.printCorrs(sectors,sectorPrices,highcorrs,highpairs)

        newStocks=[]
        newStocks2=[]
        for stock in highpairs:
            try:
                pair = stock.split(',')
                newStocks.append(pair[0])
                newStocks2.append(pair[1])
            except:""

        #recalculating correlations with highest correlating stocks
        sectors = []
        sectorsTxt = ['High Pair Set 1','High Pair Set 2']
        sectors.append(newStocks)
        sectors.append(newStocks2)


        print "sectors: ", sectors
        agentPool = Pool(self.numHighAgents)
        sectorPrices = agentPool.map(self.agentGetPricesRef,i.izip(sectors,sectorsTxt,i.repeat(self.startDate),i.repeat(self.endDate)))
        #getSectorPrices(sectors,sectorsTxt,startDate,endDate)

        #print "sectorPrices: ", sectorPrices

        highcorrs,highpairs,highidxs,fulldata=self.findHighCorrs(sectors,sectorPrices,numLags,fulldata)

        highidx=highidxs[0]

        #printing all the highest correlations
        self.printCorrs(sectors,sectorPrices,highcorrs,highpairs)

        self.printfull(fulldata)
        #self.findLeader(sectorPrices[highidx[0][0]][highidx[0][1]], sectorPrices[highidx[1][0]][highidx[1][1]], highcorrs[0],highpairs[0])

        features,classes = self.getFeatures(fulldata)

        self.classify(features,classes)

    def classify(self,features,classes):
        sv = svm.SVC()
        sv.fit(features[:500],classes[:500])
        preds = sv.predict(features[500:])

        q=0
        correct=0
        for p in preds:
            if p == classes[q+500]:correct+=1
            #print "\nclass: ",classes[q+500]
            #print "feature: ",features[q+500]
            #print "svm prediction: ",p
            q+=1
        print "svm accuracy: ",correct * 1.0 / (classes.__len__() - 500)

        #Random Forest
        rf = RandomForestClassifier()
        rf.fit(features[:500],classes[:500])
        rpreds = rf.predict(features[500:])

        r=0
        correct=0
        for p in rpreds:
            if p == classes[r+500]:correct+=1
            #print "\nclass: ",classes[r+500]
            #print "feature: ",features[r+500]
            #print "rf prediction: ",p
            #print "correct: ",correct
            r+=1
        print "rf accuracy: ",correct * 1.0 / (classes.__len__() - 500)

        #DecisionTreeClassifier
        bg = BaggingClassifier()
        bg.fit(features[:500],classes[:500])
        bpreds = bg.predict(features[500:])

        s=0
        correct=0
        for p in bpreds:
            if p == classes[s+500]:correct+=1
            #print "\nclass: ",classes[s+500]
            #print "feature: ",features[s+500]
            #print "bagg prediction: ",p
            #print "correct: ",correct
            s+=1
        print "bag accuracy: ",(correct * 1.0) / (classes.__len__() - 500)

    def getFeatures(self,fulldata):
        features = []
        classes = []

        for val in fulldata:
            tmp = []
            tmp.append(val['leaderratio'])
            tmp.append(val['followerratio'])
            tmp.append(val['corr'])
            tmp.append(val['leaderscore'])
            #tmp.append(val['a0'])
            #tmp.append(val['a1'])
            #tmp.append(val['ad1'])
            #tmp.append(val['ad5'])
            #tmp.append(val['ad10'])
            #tmp.append(val['highg'][1])
            tmp.append(val['fuzzyleader'])
            tmp.append(val['brimfuzzy'])
            if (val['highg'][1]<0.06): classes.append('leader')
            else: classes.append('notleader')
            features.append(tmp)

        return features,classes

    def printfull(self,fulldata):
        lct=0
        nlct=0
        fuzzleaderct=0
        brimfuzzyct=0
        highcorrct=0
        i=0
        for value in fulldata:
            leader = 'notleader'
            if value['highg'][1]<.06:
                leader='leader'
                lct+=1
            else:
                nlct+=1
            if value['fuzzyleader'] == 1:
                fuzzleaderct+=1
            if value['brimfuzzy'] == 1:
                brimfuzzyct+=1
            if value['corr'] > .80:
                highcorrct+=1
            print value['pair'],",",leader,",",value['corr'],",",value['a0'],",",value['a1'],",",value['ad1'],",",value['ad5'],",",value['ad10'],",",value['highg'],",",value['leaderratio'],",",value['followerratio'],",",value['leads']
            i+=1
        print "full dataset size: ", fulldata.__len__()
        print "num high corrs: ", highcorrct
        print "num leaders: ", lct
        print "non-leaders: ", nlct
        print "fuzzy leaders: ", fuzzleaderct
        print "brim fuzzy leaders: ",brimfuzzyct

    def getSectorPrices(self,sectors,sectorsTxt,startDate,endDate):
        sectorPrices = []
        i=0
        for sector in sectors:
            print "Sector: " + str(sectorsTxt[i])
            prices=[]
            for ticker in sector:
                tickerPrices = y.Share(ticker).get_historical(startDate,endDate)
                cleanPrices =[]
                for j in tickerPrices: cleanPrices.append(float(j['Adj_Close']))
                prices.append(cleanPrices)
            sectorPrices.append(prices)
            i+=1
        return sectorPrices

    def findHighCorrs(self,sectors,sectorPrices,numLags,fulldata):
        highcorrs = []
        highpairs = []
        highidxs = []
        pair={}
        pair2={}
        highx = 0
        highy = 0
        a=0
        for i in sectorPrices:
            b=0
            highcorr=[0.0,0.0,0.0]
            highpair='YHOO,YHOO'
            highidx=[[0,0],[0,0]]
            for j in sectorPrices[a]:
                c=0
                for k in sectorPrices[a]:
                    if b!=c:
                        print "b,c: ",b,c
                        pair = {}
                        pair2 = {}
                        if sectorPrices[a][b].__len__() < sectorPrices[a][c].__len__(): sectorPrices[a][b]= np.random.rand(sectorPrices[a][c].__len__(),)#np.ones(sectorPrices[a][c].__len__(),dtype=np.float)
                        #print "[a][b]: ",sectorPrices[a][b]
                        if sectorPrices[a][c].__len__() < sectorPrices[a][b].__len__(): sectorPrices[a][c]= np.random.rand(sectorPrices[a][b].__len__(),)#np.ones(sectorPrices[a][b].__len__(),dtype=np.float)
                        #print "[a][c]: ",sectorPrices[a][c]
                        corr = np.corrcoef(sectorPrices[a][b],sectorPrices[a][c])[1,0]


                        #get grangers
                        gtest=[]
                        gtest2=[]
                        d=0
                        for l in sectorPrices[a][b]:
                            gtest.append((l,sectorPrices[a][c][d]))
                            gtest2.append((sectorPrices[a][c][d],l))
                            d+=1
                        print "gtest: ",gtest
                        print "gtest2: ",gtest2
                        print "numLags: ",numLags

                        s = stats.grangercausalitytests(gtest,numLags)
                        s2 = stats.grangercausalitytests(gtest2,numLags)
                        #df = stats.adfuller(gtest,numLags)
                        #df2 = stats.adfuller(gtest2,numLags)

                        pair['pair'] = sectors[a][b] + ',' + sectors[a][c]
                        pair['data'] = sectorPrices[a][b],sectorPrices[a][c]
                        pair['corr'] = corr
                        pair['leads'] = []
                        pair['highg'] = [0,1.0]
                        pair2['pair'] = sectors[a][c] + ',' + sectors[a][b]
                        pair2['data'] = sectorPrices[a][c],sectorPrices[a][b]
                        pair2['corr'] = corr
                        pair2['leads'] = []
                        pair2['highg'] = [0,1.0]

                        fscores = []
                        fscores2 = []
                        for m in range(0,numLags):
                            fscores.append(s[m+1][0]['params_ftest'][1])
                            if fscores[m]<0.06:pair['leads'].append([m,fscores[m]])
                            if fscores[m]<pair['highg'][1]:pair['highg']=[m,fscores[m]]
                            fscores2.append(s2[m+1][0]['params_ftest'][1])
                            if fscores2[m]<0.06:pair2['leads'].append([m,fscores2[m]])
                            if fscores2[m]<pair2['highg'][1]:pair2['highg']=[m,fscores2[m]]
                        print "pair: ",pair
                        print "pair2: ",pair2

                        if pair['highg'][1]<0.06 and pair2['highg'][1]<0.06:
                            pair['fuzzyleader'] = 1
                            pair2['fuzzyleader'] = 1
                        else:
                            pair['fuzzyleader'] = 0
                            pair2['fuzzyleader'] = 0

                        p1leadscore,p2leadscore,p1leadratio,p2leadratio,brimfuzzy,brimfuzzy2 = self.leadRatios(sectorPrices[a][b],sectorPrices[a][c])
                        pair['leaderscore'] = p1leadscore
                        pair['leaderratio'] = p1leadratio
                        pair['followerratio'] = p2leadratio
                        pair2['leaderscore'] = p2leadscore
                        pair2['leaderratio'] = p2leadratio
                        pair2['followerratio'] = p1leadratio
                        pair['brimfuzzy']=brimfuzzy
                        pair2['brimfuzzy']=brimfuzzy2

                        olsres = sm.OLS(sectorPrices[a][b], sectorPrices[a][c]).fit()
                        adf = ts.adfuller(olsres.resid)
                        olsres2 = sm.OLS(sectorPrices[a][c], sectorPrices[a][b]).fit()
                        adf2 = ts.adfuller(olsres2.resid)

                        pair['a0'] = adf[0]
                        pair['a1'] = adf[1]
                        pair['ad1'] = adf[4]['1%']
                        pair['ad5'] = adf[4]['5%']
                        pair['ad10'] = adf[4]['10%']
                        pair2['a0'] = adf2[0]
                        pair2['a1'] = adf2[1]
                        pair2['ad1'] = adf2[4]['1%']
                        pair2['ad5'] = adf2[4]['5%']
                        pair2['ad10'] = adf2[4]['10%']

                        if (pair['highg'][1]<0.06 and pair2['highg'][1]<0.06) and corr > highcorr[0]:
                            highcorr= [corr,pair['highg'][1],pair2['highg'][1],pair['leaderratio'],pair2['leaderratio']]
                            highpair= str(sectors[a][b] + ',' + sectors[a][c])
                            highidx=[[a,b],[a,c]]
                            #print "corr: " , corr
                            #print "highcorr: " , highcorr

                        fulldata.append(pair)
                        fulldata.append(pair2)

                        #ols = sm.OLS(gtest,gtest2)
                        #ad = stats.adfuller(ols.resid)

                        #print "ad: ",ad
                        if highpair == 0:#' , ':
                            highcorr= [corr,pair['highg'][1],pair2['highg'][1]]
                            highpair= str(sectors[a][b] + ',' + sectors[a][c])
                            highidx=[[a,b],[a,c]]

                    c+=1
                b+=1
            a+=1
            highcorrs.append(highcorr)
            highpairs.append(highpair)
            highidxs.append(highidx)
        return highcorrs,highpairs,highidxs,fulldata

    def printCorrs(self,sectors,sectorPrices,highcorrs,highpairs):
        i=0
        plt=plot
        for sec in sectors:
            try:
                pairs=highpairs[i].split(',')
                plt.plot(sectorPrices[i][sec.index(pairs[0])], label=str(pairs[0]), linewidth=5.0)
                plt.plot(sectorPrices[i][sec.index(pairs[1])], label=str(pairs[1]), linewidth=5.0)
                corrStr = str(str(round(highcorrs[i][0],2)) + ", GC1: " + str(round(highcorrs[i][1],2)) + ", GC2: " + str(round(highcorrs[i][2],2))
                        + "\nBLA1: " + str(round(highcorrs[i][3],2)) + ", BLA2: " + str(round(highcorrs[i][4],2)))
                plt.title(str(pairs[0] + " " + pairs[1] + " Corr: " + corrStr),fontsize=20.0)
                #plt.xticks([2,19],[self.startDate,self.endDate])
                plt.legend()
                plt.show()
            except:""
            i+=1

    def getGrangers(self,sectors,sectorPrices,highcorrs,highpairs):
        i=0
        #print "highpairs: ",highpairs
        #print "sectors: ",sectors
        grangers = {}
        grs={}
        allscores=[]
        for sec in sectors:
            #print "highpairs[i]:",highpairs[i]
            pairs=highpairs[i].split(',')
            #print "pairs: ",pairs
            #print "pairs[0]: ",pairs[0]
            #print "pairs[1]: ",pairs[1]
            #print "sec: ",sec
            grs['pair'],grs['p1'],grs['p2'],grs['corr'],grs['p1high'],grs['p2high'] = pairs,pairs[0],pairs[1],highcorrs[i],[0,1.0],[0,1.0]#storing dictionary of info
            grs['p1leads'] = []#keeps track of leader/lag combo
            grs['p2leads'] = []
            gtest=[]
            gtest2=[]
            k=0
            for j in sectorPrices[i][sec.index(pairs[0])]:
                gtest.append((j,sectorPrices[i][sec.index(pairs[1])][k]))
                gtest2.append((sectorPrices[i][sec.index(pairs[1])][k],j))
                k+=1
            #print "gtest:",gtest
            s = stats.grangercausalitytests(gtest,10)
            s2 = stats.grangercausalitytests(gtest2,10)

            fscores = []
            fscores2 = []
            for l in range(0,10):
                fscores.append(s[l+1][0]['params_ftest'][1])
                if fscores[l]<0.09:grs['p1leads'].append([l,fscores[l]])

                if fscores[l]<grs['p1high'][1]:grs['p1high']=[l,fscores[l]]
                fscores2.append(s2[l+1][0]['params_ftest'][1])
                if fscores2[l]<0.09:grs['p2leads'].append([l,fscores2[l]])
                if fscores[l]<grs['p2high'][1]:grs['p2high']=[l,fscores2[l]]
            #print "fscores: ",fscores
            #print "fscores2: ",fscores2
            allscores.append(fscores)
            allscores.append(fscores2)
            #print "grs: ",grs
            grangers[i]=grs
            i+=1
        #print "allscores: ",allscores
        print "grangers: ",grangers
        print "grangers.__len__(): ",grangers.__len__()


    def findLeader(self,A=[],B=[],corrAB=0,highpair=''):
        pairs=highpair.split(',')
        length = min(len(A),len(B))
        #print pairs[0], ":", A
        #print pairs[1], ":", B
        #print "length: ", length
        corr=0.0
        corrs=[]
        prevCorr=0.0
        deltasA=[]
        deltasB=[]
        deltaA=0.0
        deltaB=0.0
        prevA=0.0
        prevB=0.0
        for i in range(length):
            if i>=0: #prime the delta vals
                corr=np.corrcoef(A[i-30:i],B[i-30:i])[1,0]
                deltaA=A[i]/A[i-1]
                deltaB=B[i]/B[i-1]
                deltasA.append(deltaA)
                deltasB.append(deltaB)
                #print "prevCorr: ", prevCorr
                #print "corr: ", corr
                #print "corrAB: ", corrAB
                #print "deltaA: ", deltaA
                #print "deltaB: ", deltaB
                prevCorr=corr
                #print "i: ", i
        plt=plot
        plt.plot(deltasA, label=str(pairs[0] + " Deltas"), linewidth=5.0)
        plt.plot(deltasB, label=str(pairs[1] + " Deltas"), linewidth=5.0)
        plt.title(str(pairs[0] + " " + pairs[1] + " Deltas"),fontsize=20.0)
        plt.xticks([2,19],[self.startDate,self.endDate])
        plt.legend()
        plt.show()
        gtest=[]
        gtest2=[]
        k=0
        for i in deltasA:
            gtest.append((i,deltasB[k]))
            gtest2.append((deltasB[k],i))
            k+=1
        #print "gtest:",gtest
        s = stats.grangercausalitytests(gtest,5)
        #print "s:",s
        #print "gtest2:",gtest2
        s2 = stats.grangercausalitytests(gtest2,5)
        #print "s2:",s2

        fscores = []
        fscores2 = []
        for i in range(0,5):
            fscores.append(s[i+1][0]['params_ftest'][1])
            fscores2.append(s2[i+1][0]['params_ftest'][1])
        #print "fscores: ",fscores
        #print "fscores2: ",fscores2

        return

    def leadRatios(self,p1,p2):
        diff = 0.0
        diffs = []
        #diffavg = np.abs((np.sum(p1) - np.sum(p2)) / p1.__len__()) * 1.0
        diffavg = 0.0
        move = 0.0
        p1move = 0.0
        p2move = 0.0
        p1moves = []
        p2moves = []
        p1diff = 0.0
        p2diff = 0.0

        if ((p1.__len__() != p2.__len__()) or (p1.__len__() < 6) or (p2.__len__() < 6)):
            print "Array Lengths must match and be great than 6 and exact same size"
            return 0.0,0.0,0.0,0.0
        brimfuzzy = 0
        brimfuzzy2 = 0
        for i in range(5,p1.__len__()):
            ##print "i: ",i
            diffavg = np.abs((np.sum(p1[i-5:i-1]) / 5.0) - (np.sum(p2[i-5:i-1]) / 5.0))
            #print "diffavg: ",diffavg
            diff = np.abs(p1[i] - p2[i]) * 1.0
            #print "diff: ",diff
            diffs.append(diff)
            move = np.abs(diff - diffavg) * 1.0
            #print "move: ", move
            p1diff = np.abs(p1[i] - p1[i-1]) * 1.0
            p2diff = np.abs(p2[i] - p2[i-1]) * 1.0
            #print "p1diff: ",p1diff
            #print "p2diff: ",p2diff
            if (p1diff == 0.0): p1move=0.0
            else: p1move = move / (p1diff * 1.0)
            if (p2diff == 0.0): p2move=0.0
            else: p2move = move / (p2diff * 1.0)
            #print "p1move: ",p1move
            #print "p2move: ",p2move
            p1moves.append(p1move)
            p2moves.append(p2move)
        #print "\n\np1: ",p1
        #print "p2: ",p2
        #print "diffs: ",diffs
        #print "p1moves: ",p1moves
        #print "p2moves: ",p2moves
        p1leadscore = (np.sum(p1moves) / p1.__len__())
        p2leadscore = (np.sum(p2moves) / p2.__len__())
        p1leadratio = (np.sum(p1moves) / p1.__len__()) / (np.sum(p2moves) / p2.__len__())
        p2leadratio = (np.sum(p2moves) / p2.__len__()) / (np.sum(p1moves) / p1.__len__())
        if(p1leadratio < 1.1 and p1leadratio > .9):#FIXME, no movements could be detected as leader
            brimfuzzy+=1
        if(p2leadratio < 1.05 and p2leadratio > .95):
            brimfuzzy2+=1


        return p1leadscore,p2leadscore,p1leadratio,p2leadratio,brimfuzzy,brimfuzzy2

#Python won't serialize a method inside a class for pooling. This allows me to do it outside of the class.
def agentGetPrices(sector,sectorTxt,startDate,endDate):
    print sectorTxt, "Agent is running for dates:", startDate, "-", endDate
    prices=[]
    for ticker in sector:
        tickerPrices = y.Share(ticker).get_historical(startDate,endDate)
        cleanPrices =[]
        for j in tickerPrices: cleanPrices.append(float(j['Adj_Close']))
        prices.append(cleanPrices)
    return prices

def agentGetPricesRef(arg):
    return agentGetPrices(*arg)

def main():
    jan = MultiagentSystem('2014-05-01','2016-04-30')
    #feb = MultiagentSystem('2016-02-01','2016-02-29')
    #mar = MultiagentSystem('2016-03-01','2016-03-31')
    #months = [august,september,october,november,december]
    months = [jan]
    for i in months: i.runAgents()


if __name__ == "__main__": main()


b,c:  0 1
gtest:  [(58.369999, 88.440002), (59.73, 89.099998), (60.580002, 90.120003), (60.630001, 89.790001), (60.209999, 89.580002), (60.439999, 89.650002), (60.049999, 89.349998), (60.09, 90.120003), (59.849998, 90.739998), (60.380001, 90.699997), (60.07, 90.150002), (60.360001, 90.260002), (60.450001, 90.239998), (59.459999, 90.110001), (59.98, 89.709999), (60.16, 90.330002), (59.990002, 89.800003), (61.560001, 90.839996), (61.18, 89.75), (61.759998, 89.709999), (63.119999, 90.260002), (62.700001, 89.709999), (62.450001, 89.150002), (61.619999, 89.43), (60.18, 88.300003), (60.32, 88.75), (59.060001, 88.529999), (58.849998, 88.639999), (59.52, 89.010002), (59.25, 88.639999), (58.73, 88.839996), (58.91, 87.660004), (58.029999, 87.32), (58.119999, 87.269997), (56.970001, 87.269997), (55.419998, 85.199997), (56.049999, 85.720001), (55.93, 84.959999), (57.0, 85.556278), (57.740002, 85.655664), (57.799999, 85.119011), (57.68, 85.119011), (58.209999, 86.232063), (56.98, 84.164963), (57.04